In [7]:
# -------------------------------------------------------------
# Algorithm: Wumpus_Inference
# -------------------------------------------------------------

# Define a helper to get valid neighbors within grid
def neighbors(x, y, N):
    adj = []
    if x > 1: adj.append((x - 1, y))
    if x < N: adj.append((x + 1, y))
    if y > 1: adj.append((x, y - 1))
    if y < N: adj.append((x, y + 1))
    return adj


# -------------------------------------------------------------
# Step 1: Initialize World Grid
# -------------------------------------------------------------
def initialize_world(N):
    Grid = {}
    for x in range(1, N + 1):
        for y in range(1, N + 1):
            Grid[(x, y)] = {
                "S": None,  # Stench
                "B": None,  # Breeze
                "P": None,  # Pit
                "W": None,  # Wumpus
                "V": False, # Visited
                "G": None   # Glitter (unused here)
            }
    return Grid


# -------------------------------------------------------------
# Step 2–8: Wumpus Inference Engine
# -------------------------------------------------------------
def Wumpus_Inference(N, Percepts):
    Grid = initialize_world(N)
    Path = [(1, 1), (2, 1), (1, 2), (2, 2), (3, 2)]

    print("=== WUMPUS WORLD INFERENCE START ===")
    print(f"Grid size: {N}x{N}")
    print("Path:", Path, "\n")

    for step, (x, y) in enumerate(Path, start=1):
        print(f"Agent is on: ({x}, {y})")

        # (a) Mark visited
        Grid[(x, y)]["V"] = True

        # (b) Read percepts
        if (x, y) in Percepts:
            Grid[(x, y)]["S"] = Percepts[(x, y)]["S"]
            Grid[(x, y)]["B"] = Percepts[(x, y)]["B"]

        # (c) Apply Rule 5: Safe cell means no Pit/Wumpus
        Grid[(x, y)]["P"] = False
        Grid[(x, y)]["W"] = False

        # (d) Apply inference rules
        for (i, j) in neighbors(x, y, N):
            # Rule2 & Rule1 (Breeze rules)
            if Grid[(x, y)]["B"] == True:
                if Grid[(i, j)]["P"] is None:
                    Grid[(i, j)]["P"] = "Possible"
            elif Grid[(x, y)]["B"] == False:
                Grid[(i, j)]["P"] = False

            # Rule3 & Rule4 (Stench rules)
            if Grid[(x, y)]["S"] == True:
                if Grid[(i, j)]["W"] is None:
                    Grid[(i, j)]["W"] = "Possible"
            elif Grid[(x, y)]["S"] == False:
                Grid[(i, j)]["W"] = False

        # (e) Display current KB snapshot (simplified grid)
        display_grid(Grid, N)
        print()

    # ---------------------------------------------------------
    # Step 9: Evaluate final entailments
    # ---------------------------------------------------------
    pit12 = Grid[(1, 2)]["P"]
    wumpus22 = Grid[(2, 2)]["W"]

    print("=== Final Entailment Results ===")
    print(f"Query1: Pit(1,2) ⇒ {pit12}")
    print(f"Query2: Wumpus(2,2) ⇒ {wumpus22}")
    print("=================================\n")

    return Grid


# -------------------------------------------------------------
# Utility function to display grid in readable format
# -------------------------------------------------------------
def display_grid(Grid, N):
    for y in range(N, 0, -1):  # print from top row down
        row = []
        for x in range(1, N + 1):
            cell = Grid[(x, y)]
            if not cell["V"]:
                row.append("     ")  # blank if not visited
                continue

            parts = []
            # Add percept info symbols
            if cell["S"] == False: parts.append("~S")
            elif cell["S"] == True: parts.append("S")

            if cell["B"] == False: parts.append("~B")
            elif cell["B"] == True: parts.append("B")

            # Pit
            if cell["P"] == False: parts.append("~P")
            elif cell["P"] == "Possible": parts.append("P?")
            elif cell["P"] == True: parts.append("P")

            # Wumpus
            if cell["W"] == False: parts.append("~W")
            elif cell["W"] == "Possible": parts.append("W?")
            elif cell["W"] == True: parts.append("W")

            row.append(",".join(parts))
        print("   ".join(f"{c:10}" for c in row))
    print()


# -------------------------------------------------------------
# Run the program with your example percepts
# -------------------------------------------------------------
if __name__ == "__main__":
    # Given percepts from your rules
    Percepts = {
        (1, 1): {"S": False, "B": False, "G": False},
        (2, 1): {"S": False, "B": True,  "G": False},
        (1, 2): {"S": True,  "B": False, "G": False}
    }

    Wumpus_Inference(N=4, Percepts=Percepts)


=== WUMPUS WORLD INFERENCE START ===
Grid size: 4x4
Path: [(1, 1), (2, 1), (1, 2), (2, 2), (3, 2)] 

Agent is on: (1, 1)
                                                 
                                                 
                                                 
~S,~B,~P,~W                                       


Agent is on: (2, 1)
                                                 
                                                 
                                                 
~S,~B,~P,~W   ~S,B,~P,~W                          


Agent is on: (1, 2)
                                                 
                                                 
S,~B,~P,~W                                       
~S,~B,~P,~W   ~S,B,~P,~W                          


Agent is on: (2, 2)
                                                 
                                                 
S,~B,~P,~W   ~P,~W                               
~S,~B,~P,~W   ~S,B,~P,~W                          


Agent i